In [1]:
# NOTE: these are not optimized (a new connection is created every time) but should work fine for initial testing

In [2]:
import concurrent.futures
import threading
import psycopg2

In [3]:
# insert data for site
def insert_site_data(domain, robots_content, crawl_delay, sitemap_content):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("\nInserting values into site")
    cur = conn.cursor()
    cur.execute("INSERT into crawldb.site (domain, robots_content, crawl_delay, sitemap_content) VALUES (%s, %s, %s, %s)", (domain, robots_content, crawl_delay, sitemap_content))
    
    cur.close()
    conn.close()
    

# testing
# insert_site_data("http://bbbb.si", "robots content string", 5, "sitemap content string")
# insert_site_data("http://ccccc.si", "robots content string", 2, "sitemap content string")
# insert_site_data("http://ddd.si", "robots content string", 5.5, "sitemap content string")


Inserting values into site


In [5]:
# insert url into frontier - inserts into table 'page' with the page_type_code value set to FRONTIER
def insert_page_into_frontier(domain, url):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("Getting site ID")
    cur = conn.cursor()

    # get id of domain url
    cur.execute("SELECT id FROM crawldb.site WHERE domain= %s", (domain,))
    site_id = cur.fetchone()[0]
    
    print("Site ID: {}".format(site_id))

    print("Inserting url into frontier")

    cur.execute("INSERT into crawldb.page (site_id, url, page_type_code) VALUES (%s, %s, 'FRONTIER')", (site_id, url))
    
    cur.close()
    conn.close()


# testing
# insert_page_into_frontier("http://bbbb.si", "https://bbbb.si/page2")
# insert_page_into_frontier("http://bbbb.si", "https://bbbb.si/page3")
# insert_page_into_frontier("http://ccccc.si", "http://ccccc.si/page1")
# insert_page_into_frontier("http://ccccc.si", "http://ccccc.si/page2")
# insert_page_into_frontier("http://ccccc.si", "http://ccccc.si/page3")

Getting site ID
Site ID: 4
Inserting url into frontier


In [7]:
# when site is crawled, update table 'page' with obtained data
def update_page_data(url, page_type_code, html_content, content_hash, http_status_code, accessed_time):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("\nInserting values into page")
    cur = conn.cursor()
    cur.execute("UPDATE crawldb.page SET page_type_code= %s, html_content= %s, content_hash = %s, http_status_code= %s, accessed_time= %s \
                WHERE url= %s", (page_type_code, html_content, content_hash, http_status_code, accessed_time, url))
    
    cur.close()
    conn.close()
    
# testing
# update_page_data("https://bbbb.si/asdasdas/asdas", "HTML", "<head>asdasdas</head>", "aphufogfdfakjnlskxcj", 200, '2022-07-01 00:00:00')
# update_page_data("https://bbbb.si/page2", "HTML", "<head>page 2 bbbb</head>", "oaisdjoaisjdojasodj", 200, '2022-07-01 00:00:00')
# update_page_data("http://ccccc.si/page1", "HTML", "<head>page1 cccc</head>", "zfzzfzflnjasdln", 200, '2022-07-01 00:00:00')



Inserting values into page


In [8]:
# get url of first page from frontier
def get_first_page_from_frontier():
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("Getting page url")
    cur = conn.cursor()

    # get url of first page with code set to FRONTIER
    cur.execute("SELECT url FROM crawldb.page WHERE page_type_code = 'FRONTIER'")
    page_url = cur.fetchone()[0]
    
    print(page_url)
    
    cur.close()
    conn.close()

get_first_page_from_frontier()

Getting page url
https://bbbb.si/page3
